In [0]:
from pyspark.sql.functions import col, regexp_replace, when, lit, max as spark_max, round
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, DecimalType, TimestampType

path = "/Volumes/workspace/raw-zone/taxi_verde/"
arquivos = dbutils.fs.ls(path)

# Dicionário para renomear colunas
renomear_colunas = {
    "vendorid": "cod_motorista",
    "lpep_pickup_datetime": "dt_hr_inicio",
    "lpep_dropoff_datetime": "dt_hr_fim",
    "passenger_count": "qtd_pessoas",
    "trip_distance": "dist_percorrida",
    "ratecodeid": "cod_taxa",
    "store_and_fwd_flag": "ind_armazenamento",
    "pulocationid": "cod_bairro_origem",
    "dolocationid": "cod_bairro_destino",
    "payment_type": "tipo_pagamento",
    "trip_type": "tipo_viagem",
    "fare_amount": "vlr_taxa_corrida",
    "extra": "vlr_taxa_extra",
    "mta_tax": "vlr_taxa_mta",
    "tip_amount": "vlr_troco",
    "tolls_amount": "vlr_pedagio",
    "ehail_fee":"vlr_gorjeta",
    "improvement_surcharge": "cod_taxa_melhoria",
    "total_amount": "vlr_total",
    "congestion_surcharge": "vlr_taxa_congestao"
}

# Dicionário de tipos das colunas
tipos_colunas = {
    "cod_motorista": StringType(),
    "dt_hr_inicio": TimestampType(),
    "dt_hr_fim": TimestampType(),
    "qtd_pessoas": IntegerType(),
    "dist_percorrida": DecimalType(10, 2),
    "cod_taxa": IntegerType(),
    "ind_armazenamento": StringType(),
    "cod_bairro_origem": StringType(),
    "cod_bairro_destino": StringType(),
    "tipo_pagamento": StringType(),
    "tipo_viagem": IntegerType(),
    "vlr_taxa_corrida": DecimalType(10, 2),
    "vlr_taxa_extra": DecimalType(10, 2),
    "vlr_taxa_mta": DecimalType(10, 2),
    "vlr_troco": DecimalType(10, 2),
    "vlr_pedagio": DecimalType(10, 2),
    "vlr_gorjeta": DecimalType(10, 2),
    "cod_taxa_melhoria": IntegerType(),
    "vlr_total": DecimalType(10, 2),
    "vlr_taxa_congestao": DecimalType(10, 2)
}

# Garantir que não haverá duplicatas - sempre reprocessa todos os dados
spark.sql("DROP TABLE IF EXISTS `trusted-zone`.tb_corrida_taxi_verde")

# Lista para armazenar DataFrames processados
dfs = []

# Processar cada arquivo parquet encontrado
for arquivo in arquivos:
    if arquivo.name.endswith(".parquet"):
        # Ler o arquivo
        df = spark.read.parquet(arquivo.path)

        # Padronizar nomes das colunas para minúsculo
        df = df.toDF(*[c.lower() for c in df.columns])

        # Renomear colunas usando dicionário
        for coluna_antiga, coluna_nova in renomear_colunas.items():
            if coluna_antiga in df.columns:
                df = df.withColumnRenamed(coluna_antiga, coluna_nova)

        # Converter todas as colunas para string para facilitar limpeza
        df = df.select([col(c).cast(StringType()).alias(c) for c in df.columns])

        # Substituir vírgulas por pontos nos valores decimais
        df = df.select([regexp_replace(col(c), ",", ".").alias(c) for c in df.columns])

        # Converter para os tipos corretos
        for coluna in df.columns:
            if coluna in tipos_colunas:
                tipo_desejado = tipos_colunas[coluna]
                
                if isinstance(tipo_desejado, (IntegerType, LongType)):
                    # Para inteiros: converte via double primeiro para aceitar "1.0"
                    df = df.withColumn(
                        coluna,
                        when(col(coluna).rlike(r"^[0-9]+(\.[0-9]+)?$"), 
                             col(coluna).cast("double").cast(tipo_desejado))
                        .otherwise(None)
                    )
                elif isinstance(tipo_desejado, (DecimalType, DoubleType)):
                    # Para decimais: converte e força arredondamento para 2 casas
                    df = df.withColumn(
                        coluna,
                        when(col(coluna).rlike(r"^[0-9]+(\.[0-9]+)?$"), 
                             round(col(coluna).cast("double"), 2).cast(tipo_desejado))
                        .otherwise(None)
                    )
                elif isinstance(tipo_desejado, TimestampType):
                    # Para timestamps
                    df = df.withColumn(coluna, col(coluna).cast(TimestampType()))
                else:
                    # Para strings
                    df = df.withColumn(coluna, col(coluna).cast(StringType()))

        # Adicionar coluna identificadora da origem
        df = df.withColumn("origem_taxi", lit("taxi_verde"))

        # Adicionar DataFrame processado à lista
        dfs.append(df)

# Unir todos os DataFrames se houver arquivos processados
if dfs:
    df_final = dfs[0]
    for df_individual in dfs[1:]:
        df_final = df_final.unionByName(df_individual)

    # Salvar tabela final com todos os dados
    df_final.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("`trusted-zone`.tb_corrida_taxi_verde")

    display(df_final)